# Semester assignment

In [ ]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
from matplotlib import gridspec

#Setting new parameters for plotting
newParams = {'figure.figsize'  : (12, 6),  # Figure size
             'figure.dpi'      : 100,      # figure resolution
             'axes.titlesize'  : 20,       # fontsize of title
             'axes.labelsize'  : 11,       # fontsize of axes labels
             'axes.linewidth'  : 2,        # width of the figure box lines
             'lines.linewidth' : 1,        # width of the plotted lines
             'savefig.dpi'     : 200,      # resolution of a figured saved using plt.savefig(filename)
             'ytick.labelsize' : 11,       # fontsize of tick labels on y axis
             'xtick.labelsize' : 11,       # fontsize of tick labels on x axis
             'legend.fontsize' : 12,       # fontsize of labels in legend
             'legend.frameon'  : True,     # activate frame on lengend?
            }
plt.rcParams.update(newParams) # Set new plotting parameters

plt.style.use('default')

Boundary conditions:
\begin{equation}
    V(x = 0, y) = 0, \\
    V(x = L, y) = 0, \\
    V(x, y = 0) = 0, \\
    V(x, y = L) = V_0 (x)
\end{equation}

Solving Laplace equation ($\nabla^2 V = 0$) with these boundary condition gives:

\begin{align}
    V(x, y) =& \sum_{n=1}^{\infty} c_n \text{sinh}(n \pi \xi_y ) \text{sin}(n  \pi \xi_x),\\
    c_n =& \frac{2}{\text{sinh}(n \pi)} \int_0^1 \text{d} \xi_x V_0 (\xi_x L) \text{sin}(n \pi \xi_x),
\end{align}

where $\xi_x = x/L$ and $\xi_y = y/L$ are dimensionless variables.

## Finding $V(x, y)$

In [ ]:
########################################################
### Defining functions for calculations and plotting ###
########################################################


def c_n(x, V_0, n, int_method="riemann"):
    '''Returns the fourier coefficient nr. n. Default integration method is the left Riemann sum.
    INPUT: x - values along x-axis (1D or 2D array),
        V_0 - initial condition at at y=L (1D array), 
        n - fourier coefficient number (integer),
        int_method: method of numerical integration. Can be either "riemann", "trapz" or "simps" (string).
    OUTPUT: fourier coefficient n (float)
    '''
    dx = L/N
    if int_method.lower() == "riemann": #implement left riemann sum method of integration
        integral = np.sum(V_0 * np.sin(n*np.pi*x), axis=1) * dx
    elif int_method.lower() == "trapz":
        integral = integrate.trapz(V_0 * np.sin(n*np.pi*x), dx=dx)
    elif int_method.lower() == "simps":
        integral = integrate.simps(V_0 * np.sin(n*np.pi*x), dx=dx)
    else:
        print("Integration method was not recognized.\n" +
            "Please enter 'riemann' for left Riemann summation, 'trapz' for trapezoidal rule or 'simps' for Simpsons rule.")
        return None
    return 2 * integral / np.sinh(n*np.pi) 


def potential(x, y, V_0, n, int_method="riemann"):
    '''Calculates the potential given by V_0(x).
    INPUT: x and y - positions to evaluate (meshgrids/2D arrays), 
        V_0 - initial condition at at y=L (1D array), 
        n - number of terms in the sum  (integer).
    OUTPUT: V - calculated potential (meshgrid/2D array)
    '''
    V = np.zeros_like(x)  #Initializing an NxN array for V
    norm_diff = np.zeros(n)  #Initializing an array for the differences of the norms to evaluate convergence.
    init_diff = np.zeros(n)  #Initializing an array for the differences of the edge potential at y=L and the inital potential.
    for i in range(1, n):
        ci = c_n(x, V_0, i, int_method)  #Calculating the fourier coefficient.
        Vi = ci*np.sinh(i*np.pi*y)*np.sin(i*np.pi*x)  #Calculating term i in the sum of V.
        prev_norm = np.linalg.norm(V) 
        V += Vi 
        norm_diff[i] = np.linalg.norm(V) - prev_norm  #Calculating the norm difference of the potential between steps.
        init_diff[i] = np.linalg.norm((V_0 - V[-1,:]))  #Calculating the difference between the edge potential at y=L and the inital potential.
    return V, norm_diff, init_diff


def init_pot_1(x, m=1):
    '''Initial potential of y=L number 1.'''
    return np.sin(m*np.pi*x)


def init_pot_2(x):
    '''Initial potential of y=L number 2.'''
    return (1 - (x - 0.5)**4)


def init_pot_3(x):
    '''Initial potential of y=L number 3.'''
    return np.heaviside(x - L*0.5, 1)*np.heaviside(L*0.75 - x, 1)


def plot_potential(x, y, V, pot_num=None):
    '''Plotting a colormap of the potential V with the edge potentials on the corresponding side of the tube.'''
    
    # Creating meshgrids for the colormap
    xx, yy = np.meshgrid(x, y)
    
    # Plotting a colormap of the potential V(x, y) with the edge potentials on the corresponding side of the tube.
    fig = plt.figure("V(x,y) in infinite, square hollow tube", figsize=(12,7))

    #Creating a gridspec to place the potential plots in the correct way.
    gs = gridspec.GridSpec(10, 5, figure=fig)   

    # Plotting V(x,y=L) as a function of x, should be similar to V_0(x).
    plt.subplot(gs.new_subplotspec((1, 1), colspan=3, rowspan=2))  
    plt.plot(x, V[len(y)-1,:], label="V(x, y=L)")
    plt.ylabel(r"$V(x, y=L)/V_c$")

    # Plotting V(x=0,y) as a function of y flipped to match the colormap. Should be 0.
    plt.subplot(gs.new_subplotspec((3, 0), rowspan=5))   
    plt.plot(V[:,0], y, label="V(x = 0, y)")
    plt.ylabel(r"$\xi_y$", fontsize=14)
    plt.xlabel(r"$V(x=0, y)/V_c$")

    # Plotting a colormap of the potential
    ax = plt.subplot(gs.new_subplotspec((3, 1), colspan=3, rowspan=5))   
    ax.axes.get_xaxis().set_visible(False) 
    ax.axes.get_yaxis().set_visible(False) 

    # Set colour interpolation and colour map
    colorinterpolation = 50
    colourMap = plt.cm.jet 

    # Configure the contour
    im = plt.contourf(xx, yy, V, colorinterpolation, cmap=colourMap)

    # Plotting V(x=L,y) as a function of y, flipped to match the colormap. Should be 0. 
    plt.subplot(gs.new_subplotspec((3, 4), rowspan=5))   
    plt.plot(V[:,-1], y, label="V(x=L, y)")
    plt.xlim(-0.05, 0.05)
    plt.xlabel(r"$V(x=L, y)/V_c$")

    # Plotting V(x,y=0) as a function of x. Should be 0.
    plt.subplot(gs.new_subplotspec((8, 1), colspan=3, rowspan=2))   
    plt.plot(x, V[0,:], label="V(x, y=0)")
    plt.xlabel(r"$\xi_x$",fontsize=14)
    plt.ylabel(r"$V(x, y=0)/V_c$")

    fig.suptitle(f'V(x, y) with initial potential {pot_num}', fontsize=16)

    plt.tight_layout(pad=0.3, w_pad=0.3, h_pad=0.3)

    # Adding the colorbar on the right side of the entire plot.
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    cb = fig.colorbar(im, cax=cbar_ax)
    cb.set_label("V")

    # Saving the plot as .png and showing the result in the plot window.
    #plt.savefig(f"pot{pot_num}.pdf")
    plt.show()
    
    
def plot_convergence(n, diff_1, diff_2, diff_3, n_end_1, n_start_2, n_end_2, n_start_3, n_end_3, type="norm_diff"):
    '''Plotting the convergence, either norm_diff or init_diff.'''
    
    if type.lower() == "norm_diff": 
        ylabel = "Norm difference of V"
        diff_1_normalized = diff_1 / np.sum(diff_1)  # Normalize norm_diff
        diff_2_normalized = diff_2 / np.sum(diff_2)
        diff_3_normalized = diff_3 / np.sum(diff_3)
        cum_diff_1 = np.cumsum(diff_1_normalized)  # Calculating the cumulative sum of the normalized norm_diff
        cum_diff_2 = np.cumsum(diff_2_normalized)
        cum_diff_3 = np.cumsum(diff_3_normalized)
    elif type.lower() == "init_diff": ylabel = r"Norm difference of V$_0$(x) and V(x, y=L)"
    elif type.lower() == "cum_diff": ylabel = "Cumulative norm difference of V"
    else: ylabel = ""
    
    # Plotting the rate of convergence of V(x,y)
    plt.figure(f"Convergence {type}", figsize=(18, 4))
    #plt.suptitle(f"Convergence of V(x,y), {type}", fontsize=16)

    # Plotting the rate of convergence for initial potential 1: V_0(x) = sin(m*pi*x)
    plt.subplot(131)
    plt.title(r"$V_0(x) = V_c$ sin($m \pi x$), m = 1")
    if type.lower() == "init_diff": 
        plt.plot(np.arange(n_end_1+1), diff_1[:n_end_1+1], "r-" ,label="Norm difference (normalized)")
        plt.ylim(-0.5, 10.5)
    if type.lower() == "norm_diff": 
        plt.plot(np.arange(n_end_1+1), diff_1_normalized[:n_end_1+1], "r-" ,label="Norm difference (normalized)")
        plt.plot(np.arange(n_end_1+1), cum_diff_1[:n_end_1+1], "r--", label="Cumulative norm difference")
    plt.xlabel("n")
    plt.ylabel(ylabel)
    plt.legend()

    # Plotting the rate of convergence for initial potential 2: V_0(x) = V_0(x) = 1 - (x - 1/2)**4
    plt.subplot(132)
    plt.title(r"$V_0 (x) = V_c \left[1 - (x - 1/2)^4 \right]$")
    if type.lower() == "init_diff": 
        plt.plot(np.arange(n_start_2, n_end_2+1), diff_2[n_start_2:n_end_2+1], "b-", label="Norm difference (normalized)")
    if type.lower() == "norm_diff": 
        plt.plot(np.arange(n_start_2, n_end_2+1), diff_2_normalized[n_start_2:n_end_2+1], "b-", label="Norm difference (normalized)")
        plt.plot(np.arange(n_start_2, n_end_2+1), cum_diff_2[n_start_2:n_end_2+1], "b--", label="Cumulative norm difference")
    plt.xlabel("n")
    plt.legend()

    # Plotting the rate of convergence for initial potential 3: V_0(x) = theta(x-L/2)*theta(3L/4 - x)
    plt.subplot(133)
    plt.title(r"$V_0 (x) = V_c \theta(x-L/2) \theta(3L/4 - x)$")
    if type.lower() == "init_diff": plt.plot(np.arange(n_start_3, n_end_3+1), diff_3[n_start_3:n_end_3+1], "g-", label="Norm difference (normalized)")
    if type.lower() == "norm_diff": 
        plt.plot(np.arange(n_start_3, n_end_3+1), diff_3_normalized[n_start_3:n_end_3+1], "g-", label="Norm difference (normalized)")
        plt.plot(np.arange(n_start_3, n_end_3+1), cum_diff_3[n_start_3:n_end_3+1], "g--", label="Cumulative norm difference")
    plt.xlabel("n")
    plt.legend()

    # Saving the figure as .png and showing the result.
    #plt.savefig(f"convergence_{type}.png")
    plt.show()
    
    
def el_field(V):
    '''Calculating the electric field and absolute value of a potential V.
    INPUT: V - potential (2D array)
    OUTPUT: E - Electric field (array with two elements made up of 2D arrays)
        E_abs - Absolute square of the electric field (float)
    '''
    E = np.gradient(V)
    E_abs = np.sqrt(E[0]**2 + E[1]**2)*L/N
    return E, E_abs

In [ ]:
###################################
### Setting starting parameters ###
###################################

start = 0
L = 1
N = 1000    #number of points in x and y
n = 100    #number of fourier terms

# Creating x and y axes with N values between 0 and L
x = np.linspace(0, L, N)
y = np.linspace(0, L, N)

# Creating meshgrids for colormap of the potential
xx, yy = np.meshgrid(x, y)



##########################################################################################
### Calculating potentials and convergence parameters for different initial potentials ###
##########################################################################################

#Initial potential 1, sin(m*pi*x)
V_1, norm_diff_1, init_diff_1 = potential(xx, yy, init_pot_1(x), n)

#Initial potential 2, V_0(x) = 1 - (x - 0.5)**4
V_2, norm_diff_2, init_diff_2 = potential(xx, yy, init_pot_2(x), n)

#Initial potential 3, V_0(x) = theta(x-L/2) * theta(3L/4 - x)
V_3, norm_diff_3, init_diff_3 = potential(xx, yy, init_pot_3(x), n)

In [ ]:
# Finding the overshoots of potential V_2 and V_3
V_2_max, a_2 = V_2.max(), init_pot_2(x)[0]
V_3_max, a_3 = V_3.max(), init_pot_3(x).max()

overshoot_2 = V_2_max / a_2 - 1.0
overshoot_3 = V_3_max / a_3 - 1.0

print(f"Overshoot of V_2 = {overshoot_2:.3f} = {overshoot_2*100:.1f} % \nOvershoot of V_3 = {overshoot_3:.3f} = {overshoot_3*100:.1f} %")

In [ ]:
####################################
### Plotting the egde potentials ###
####################################

plt.figure("Initial potentials", figsize=(10,5))
ax1 = plt.subplot(311)
ax1.axes.get_xaxis().set_visible(False)  #hide x-axis
plt.plot(x, init_pot_1(x), x, V_1[len(y)-1,:], "--")
plt.ylabel(r"$\frac{V_{1}(x)}{V_0}$")

ax2 = plt.subplot(312)
ax2.axes.get_xaxis().set_visible(False)  #hide x-axis
init_2 = init_pot_2(x)
init_2[0] = init_2[-1] = 0
plt.plot(x, init_2, x, V_2[len(y)-1,:], "--")
#plt.ylim(-0.2, 1.2)
plt.ylabel(r"$\frac{V_{2}(x)}{V_0}$")

plt.subplot(313)
plt.plot(x, init_pot_3(x), x, V_3[len(y)-1,:], "--")
plt.xlabel(r"$\xi_x$")
plt.ylabel(r"$\frac{V_{3}(x)}{V_0}$")
plt.tight_layout()

#plt.savefig("initial_potentials.png")
plt.show()


In [ ]:
###############################
### Plotting the potentials ###
###############################  
    
plot_potential(x, y, V_1, 1)
plot_potential(x, y, V_2, 2)
plot_potential(x, y, V_3, 3)

## Convergence of $V(x, y)$

One way to investigate the rate of convergence for $V(x, y)$ is to look at the difference of the norm from one iteration to the next. Another way to study the convergence is to compare with the initial edge potential at $y = L$. Below, the differences are plotted as a function of the summation number n. 

In [ ]:
###############################################
### Plotting convergence of the potentials: ###
###############################################
    
plot_convergence(n, norm_diff_1, norm_diff_2, norm_diff_3, 10, 0, 30, 0, 40)
plot_convergence(n, init_diff_1, init_diff_2, init_diff_3, 99, 1, 99, 1, 99, "init_diff")

In [ ]:
#testing different methods of integration:
N = 1000

#Initial potential 2, V_0(x) = 1 - (x - 0.5)**4
V_riemn, norm_diff_riemn, init_diff_riemn = potential(xx, yy, init_pot_3(x), n)
V_trapz, norm_diff_trapz, init_diff_trapz = potential(xx, yy, init_pot_3(x), n, "trapz")
V_simps, norm_diff_simps, init_diff_simps = potential(xx, yy, init_pot_3(x), n, "simps")

In [ ]:
def plot_convergence_int_method(n, diff_1, diff_2, diff_3, type="norm_diff"):
    
    #Plotting the rate of convergence of V(x,y)
    plt.figure(f"Convergence, {type}", figsize=(20, 5))
    plt.suptitle(r"Convergence of $V(x,y)$", fontsize=16)

    # Plotting the rate of convergence for initial potential 2 calculated with left Riemann sum.
    plt.subplot(131)
    start_n = 0
    len_n = 100
    plt.plot(np.arange(start_n, start_n+len_n), diff_1[start_n:start_n+len_n], "r-" ,label="Left Riemann sum")
    #plt.plot(np.arange(start_n, start_n+len_n), np.ones(len_n)*0.05, 'k--', linewidth=0.7)
    plt.xlabel("n")
    plt.ylabel("Norm difference")
    plt.legend()

    # Plotting the rate of convergence for initial potential 2 calculated with the trapezoidal rule.
    plt.subplot(132)
    plt.plot(np.arange(start_n, start_n+len_n), diff_2[start_n:start_n+len_n], "b-", label="Trapezoidal rule")
    #plt.plot(np.arange(start_n, start_n+len_n), np.ones(len_n)*0.05, 'k--', linewidth=0.7)
    plt.xlabel("n")
    plt.legend()

    # Plotting the rate of convergence for initial potential 2 calculated with Simpson's rule.
    plt.subplot(133)
    plt.plot(np.arange(start_n, start_n+len_n), diff_3[start_n:start_n+len_n], "g-", label="Simpson's rule")
    #plt.plot(np.arange(start_n, start_n+len_n), np.ones(len_n)*0.05, 'k--', linewidth=0.7)
    plt.xlabel("n")
    plt.legend()

    # Saving the figure as .png and showing the result.
    #plt.savefig(f"convergence_{type}.png")
    plt.show()
    

#plot_convergence_int_method(n, norm_diff_riemn, norm_diff_trapz, norm_diff_simps, type="norm_diff")
#plot_convergence_int_method(n, init_diff_riemn, init_diff_trapz, init_diff_simps, type="init_diff")

## Finding $\bf{E}$$(x, y)$

To find the electric field, one takes the divergence of the potential,

\begin{equation}
    E = - \nabla V.
\end{equation}

In [ ]:
##################################
### Finding the electric field ###
##################################

#Calculating the electric field E from V, and |E|^2 to use as a color reference.
E_1, E_abs_1 = el_field(V_1)
E_2, E_abs_2 = el_field(V_2)
E_3, E_abs_3 = el_field(V_3)


# Plotting E as a streamplot.
plt.figure("Electric field", figsize=(18,4))
#plt.suptitle(r"$\bf{E}$(x, y)", fontsize=16)

# Plotting the electric field of V(x,y) with initial potential 1: V_0(x) = sin(m*pi*x)
plt.subplot(131)
plt.title(r"$V_0(x)$ = sin($m \pi x$), m = 1")
plt.streamplot(xx, yy, E_1[0], E_1[1], color=E_abs_1)
plt.xlabel(r"$\xi_x$")
plt.ylabel(r"$\xi_y$")
cb1 = plt.colorbar()
cb1.set_label(r"|$\bf{E}$|")

# Plotting the electric field of V(x,y) with initial potential 2: V_0(x) = V_0(x) = 1 - (x - 1/2)**4
plt.subplot(132)
plt.title(r"$V_0 (x) = 1 - (x - 1/2)^4$")
plt.streamplot(xx, yy, E_2[0], E_2[1], color=E_abs_2)
plt.xlabel(r"$\xi_x$")
plt.ylabel(r"$\xi_y$")
cb2 = plt.colorbar()
cb2.set_label(r"|$\bf{E}$|")

#  Plotting the electric field of V(x,y) with initial potential 3: V_0(x) = theta(x-L/2)*theta(3L/4 - x)
plt.subplot(133)
plt.title(r"$V_0 (x) = \theta(x-L/2) \theta(3L/4 - x)$")
plt.streamplot(xx, yy, E_3[0], E_3[1], color=E_abs_3)
plt.xlabel(r"$\xi_x$") #"\n"  r"$V_0 (x) = \theta(x-L/2) \theta(3L/4 - x)$")
plt.ylabel(r"$\xi_y$")
cb3 = plt.colorbar()
cb3.set_label(r"|$\bf{E}$|")

# Saving the plot as .png and showing the result in the plot window.
#plt.savefig("E_field.png")
plt.show()